### Running a single-machine cluster
When run on a single machine, this is equivalent to the dask 'processes' scheduler, with the added addition of a handy status dashboard. It's accessible by web browser via the link below. (The local cluster server can be set up and run outside the notebook for persistence.)

In [1]:
from dask import distributed

CLUSTER_SETTINGS = dict(
    # limiting this can help to avoid system hangs caused by google drivefs
    n_workers=8,
    threads_per_worker=1,
    # host=f"tcp://127.0.0.1:8786",
    dashboard_address=f"127.0.0.1:8787",
)

# cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
if 'scheduler' not in dir():
    cluster = distributed.LocalCluster(**CLUSTER_SETTINGS)
    scheduler = distributed.Client(cluster.scheduler_address)

cluster

2023-03-10 15:12:25,892 - bokeh.server.util - WARNING - Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly


LocalCluster(91b6c8cb, 'tcp://127.0.0.1:10247', workers=8, threads=8, memory=31.60 GiB)

### Creating dask dataframe objects
The following assumes that you've downloaded the specified zip archive at the specified path.

In [7]:
from dask_ops import zipfile_dask_dfs, log_to_json, trace
import eliot
from pathlib import Path
from labbench import stopwatch
import pandas as pd

def capture_summary(partition_data: dict):
    """ this could be expanded into a function that does more, but for now, just make a capture summary """

    # partition_data is the result of running zipfile.read_seamf_zipfile on a subset of files in the zip archive.
    # it takes the form of a dictionary of pandas.DataFrame objects

    partition_data['capture_summary'] = pd.DataFrame.from_dict({
        'median_rms_pfp': trace(partition_data, 'pfp', capture_statistic='mean', detector='rms').median(axis=1),
        'max_max_pfp': trace(partition_data, 'pfp', capture_statistic='max', detector='peak').max(axis=1),
        'median_mean_power': trace(partition_data, 'psd', capture_statistic='mean').median(axis=1),
        'max_max_power': trace(partition_data, 'psd', capture_statistic='mean').max(axis=1),
        'noise_figure': trace(partition_data, 'channel_metadata')['cal_noise_figure_dB'],
        'gain': trace(partition_data, 'channel_metadata')['cal_gain_dB'],
        # TODO: add 'temperature' from sweep_metadata, which isn't easy yet b/c the 
    })

    return partition_data

data_path = Path('data')/'NIT-2022-12-13.zip'

# overwrite
log_to_json('data.log')

with stopwatch('setup'):
    # scan the zipfiles to map out the file contents. the resulting is a dictionary of dask dataframes.
    # this is fast, because it hasn't loaded much data yet; other operations trigger "compute" that can
    # take a while to scrape the data out of the zip archive
    ddfs = zipfile_dask_dfs(
        data_path,
        partition_func=capture_summary,
        partition_size=100
    )

C:\Users\dkuester\AppData\Local\pdm\pdm\Cache\packages\eliot-1.14.0-py2.py3-none-any\lib\eliot\json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):
C:\Users\dkuester\AppData\Local\pdm\pdm\Cache\packages\dask-2023.3.0-py3-none-any\lib\dask\base.py:1369: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
 INFO   2023-03-10 15:15:39.101 • labbench: setup 2.564 s elapsed


In [11]:
with stopwatch('compute'):
    # head forces a compute operation
    df = ddfs['capture_summary'].head(10)
df

 INFO   2023-03-10 15:17:33.716 • labbench: compute 1.643 s elapsed


,frequency,median_rms_pfp,max_max_pfp,median_mean_power,max_max_power,noise_figure,gain
datetime,,,,,,,
2022-10-26 20:47:37.094,3.555000e+09,-86.1875,-64.81250,-155.375,-152.000,4.330333,31.440102
2022-10-26 20:47:41.699,3.565000e+09,-88.5000,-66.06250,-157.250,-154.125,4.265367,31.498794
2022-10-26 20:47:46.374,3.575000e+09,-86.2500,-64.00000,-155.750,-152.875,4.256935,31.315659
2022-10-26 20:47:51.554,3.585000e+09,-88.0625,-65.31250,-156.000,-153.875,4.031151,31.589694
2022-10-26 20:47:56.944,3.595000e+09,-82.7500,-60.12500,-150.125,-146.375,4.00401,31.71745
2022-10-26 20:48:01.623,3.605000e+09,-83.5625,-60.87500,-150.750,-147.875,3.989708,31.672517
2022-10-26 20:48:07.579,3.615000e+09,-83.8750,-64.06250,-153.250,-150.000,4.088225,31.528059
2022-10-26 20:48:12.256,3.625000e+09,-86.8750,-64.12500,-155.500,-152.625,4.056286,31.530697
2022-10-26 20:48:16.953,3.635000e+09,-79.7500,-59.28125,-151.375,-145.625,3.997318,31.689596


#### on-demand access to a subset of the data
The dask .loc accessor allows faster access to a specified subset of the data

In [ ]:
with stopwatch('compute'):
    df = (
        ddfs['capture_summary']
        .loc['2022-11-21 20:50:00':'2022-11-21 20:51:00']
        .compute(scheduler=scheduler)
    )

#### saving data in files split by time period
Dask dataframes support data save operations split output files by partition. This means that the time span of each output file can be adjusted by simply calling 'repartition'.

In [44]:
import dask

with stopwatch('compute'):
    # repartition by month divided at the start of each month
    ddf = ddfs['capture_summary'].repartition(freq='2W')
    name_func = lambda i: ddf.divisions[i].strftime('%Y-%m-%d')
    capture_op = (
        ddf
        .to_csv('data/capture_summary/*.parquet', name_function=name_func, compute=False)
    )

    ddf = ddfs['pfp'].repartition(freq='1W')
    name_func = lambda i: ddf.divisions[i].strftime('%Y-%m-%d')
    pfp_op = (
        ddf
        .to_csv('data/pfp/*.parquet', name_function=name_func, compute=False)
    )

    files_saved = dask.compute(dict(pfp=pfp_op, capture_summary=capture_op), scheduler=scheduler)

    print('wrote the following files:')
    print(files_saved)

2023-03-10 16:20:19,881 - bokeh.server.views.ws - WARNING - Failed sending message as connection was closed
